In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import math

<h1>1. Preprocesamiento</h1>

<h2>1.1 Normalización</h2>

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_0 = pd.read_csv('Data\Concrete_Data_Yeh.csv')

df_p = preprocessing.normalize(df_0, axis=0)
df = pd.DataFrame(df_p, columns=df_0.columns)
df.describe()

cement         slag       flyash        water  superplasticizer  \
count  1030.000000  1030.000000  1030.000000  1030.000000       1030.000000   
mean      0.029208     0.020274     0.020141     0.030946          0.022451   
std       0.010856     0.023672     0.023786     0.003640          0.021616   
min       0.010596     0.000000     0.000000     0.020759          0.000000   
25%       0.019984     0.000000     0.000000     0.028105          0.000000   
50%       0.028349     0.006036     0.000000     0.031531          0.023158   
75%       0.036359     0.039220     0.043969     0.032724          0.036908   
max       0.056096     0.098607     0.074372     0.042098          0.116515   

       coarseaggregate  fineaggregate          age        csMPa  
count      1030.000000    1030.000000  1030.000000  1030.000000  
mean          0.031060       0.030993     0.018259     0.028241  
std           0.002482       0.003212     0.025260     0.013172  
min           0.025571       0.023798     0.000400     0.001837  
25%           0.029754       0.029285     0.002799     0.018694  
50%           0.030903       0.031230     0.011197     0.027158  
75%           0.032863       0.033013     0.022393     0.036375  
max           0.036554       0.039768     0.145956     0.065126

<h2>1.2 Selección de variables</h2>

<h3>1.2.1 Generación de nuevas variables</h3>

In [4]:
## Generación de realciones

columnas = list(df.drop(columns=['csMPa']).columns)
for i in columnas:
    columnas_2 = [x for x in columnas if x not in i]
    for j in columnas_2:
        nombre_columna = i + '-' + j
        df[nombre_columna] = df[i] / df[j]

## Dejar varibales con valores no indeterminados

df = df[list(df.describe().replace([np.inf, -np.inf], np.nan).iloc[-1].dropna().index)]

## Generación de Exponenciales

columnas = list(df.drop(columns=['csMPa']).columns)

for i in columnas:
    nombre_columna = 'exp_' + i
    df[nombre_columna] = df[i].apply(lambda x: math.exp(x))

D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:3949: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


<h3>1.2.2 Selección por GA con AIC</h3>

In [5]:
## Definición funcion a optimizar

def aic_criterion(df):
    n_var = len(df.drop(columns=['residuos'
                                 , 'csMPa'
                                 , 'csMPa_est']).columns)
    
    n = len(df)
    
    res = sum(df['residuos'].apply(lambda x: x ** 2))
    
    return((n * (math.log(2 * math.pi) + 1 + math.log(res / (n - n_var - 1)))) + ((n_var + 1) * 2))

In [6]:
## Generar población inicial aleatoria

np.random.seed(1984)

explicativas = list(df.drop(columns=['csMPa']).columns)
df_explicativas = pd.DataFrame(np.random.randint(2,size=(500, len(explicativas))), columns=explicativas)
df_explicativas['aic'] = 200000

y = df['csMPa']
aic_col = pd.Series([])

for i in range(len(df_explicativas)):
    X = df[list(df_explicativas.iloc[i][df_explicativas.iloc[i] == 1].index)]
    X_val, X_other, y_val, y_other = train_test_split(X, y, test_size=0.80, random_state=42, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(X_other, y_other, test_size=0.40, shuffle=False)

    del X_other
    del y_other
    
    reg = LinearRegression().fit(X_train, y_train)
    y_val_est = pd.Series(reg.predict(X_val))
    
    df_val = pd.concat([X_val, y_val,y_val_est], axis=1).rename(columns={0 : 'csMPa_est'})
    df_val['residuos'] = df_val['csMPa'] - df_val['csMPa_est']
    
    df_explicativas['aic'].iloc[i] = aic_criterion(df_val)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

...

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, va

In [108]:
## Generar operadores de cruce y mutacion

def cruce(cromosoma_0, cromosoma_1, tipo_cruce):
    
    rng = np.random.default_rng()
    largo_cromosoma = len(cromosoma_0)
    rango_cromosoma = range(largo_cromosoma)
    
    if tipo_cruce == 'punto unico':
        
        punto = rng.choice(rango_cromosoma, size=1, replace=False)[0]
        
        descendencia_0 = np.concatenate((cromosoma_0[:punto], cromosoma_1[punto:]), axis = 0)
        descendencia_1 = np.concatenate((cromosoma_1[:punto], cromosoma_0[punto:]), axis = 0)
        
        return(descendencia_0, descendencia_1)
    
    elif tipo_cruce == 'dos puntos':
        
        puntos = rng.choice(rango_cromosoma, size=2, replace=False)
        punto_0 = min(puntos[0], puntos[1])
        punto_1 = max(puntos[0], puntos[1])
        
        descendencia_0 = np.concatenate((cromosoma_0[:punto_0]
                                         , cromosoma_1[punto_0:punto_1]
                                         , cromosoma_0[punto_1:]), axis = 0)
        descendencia_1 = np.concatenate((cromosoma_1[:punto_0]
                                         , cromosoma_0[punto_0:punto_1]
                                         , cromosoma_1[punto_1:]), axis = 0)
        
        return(descendencia_0, descendencia_1)
    
    elif tipo_cruce == 'uniforme':

        padre_0 = np.random.randint(2,size=(len(cromosoma_0), 1))
        padre_1 = (padre_0 - 1) * (- 1)
        
        descendencia_0 = [(padre_0[i] * cromosoma_0[i])[0] + (padre_1[i] * cromosoma_1[i])[0] for i in range(len(cromosoma_0))]
        descendencia_1 = [(padre_0[i] * cromosoma_1[i])[0] + (padre_1[i] * cromosoma_0[i])[0] for i in range(len(cromosoma_0))]
        
        return(descendencia_0, descendencia_1)

    
    
def mutacion(cromosoma_0, df, tipo_dato):
    
    rng = np.random.default_rng()
    largo_cromosoma = len(cromosoma_0)
    rango_cromosoma = range(largo_cromosoma)
    
    gen_mutacion = rng.choice(rango_cromosoma, size=1, replace=False)[0]
    
    if tipo_dato == 'binario':
        
        if cromosoma_0[gen_mutacion] == 1:
            nuevo_gen = 0
        else:
            nuevo_gen = 1
    
    elif tipo_dato == 'flotante':
        
        std_gen = np.std(np.array(df.iloc[:, gen_mutacion]))
        
        nuevo_gen = cromosoma_0[gen_mutacion] + np.random.normal(0, std_gen, 1)
        
    cromosoma_0 = np.concatenate((cromosoma_0[:gen_mutacion]
                                  ,np.array([nuevo_gen])[0]
                                  ,cromosoma_0[gen_mutacion+1:]), axis = 0)
    
    return(cromosoma_0)

In [8]:
## Generar funciones de seleccion

def seleccion(df, factor_seleccion, tipo_seleccion, tamano_torneo, tamano_elitismo):
    
    df['seleccionado'] = 0
    
    if tipo_seleccion == 'ruleta':
        
        df['dummy'] = 0
        
        max_aic = max(df['aic'])
        df['aic_escalado'] = (df['aic'] - max_aic - 1) * (-1)
            
        total_aic_esc = sum(df['aic_escalado'])
        df['prob'] = df['aic_escalado'] / total_aic_esc
        df['prob_acum'] = df.groupby(['dummy'])['prob'].cumsum()
        df['prob_acum_lag'] = df['prob_acum'].shift(1).fillna(0)
            
        while (sum(df['seleccionado']) / len(df)) < factor_seleccion:
                
            rand = np.random.uniform(0, 1)
            df['seleccionado'] =  df.apply(lambda x: 1 if ((x.prob_acum_lag < rand) 
                                                           & (rand <= x.prob_acum)) else x.seleccionado, axis = 1)

        df = df.drop(columns=['dummy','aic_escalado','prob','prob_acum','prob_acum_lag'])
                
    elif tipo_seleccion == 'torneo':
        
        n = len(df) - 1
        df['fila'] = df.index
        
        while (sum(df['seleccionado']) / len(df)) < factor_seleccion:
                
            rand = list(np.random.randint(0, n, tamano_torneo))
            
            df['en_torneo'] = df.apply(lambda x: 1 if x.fila in rand else 0, axis = 1)
            df['min_aic'] = min(df[df['en_torneo'] == 1]['aic'])
            df['seleccionado'] =  df.apply(lambda x: 1 if ((x.min_aic == x.aic)
                                                           & (x.en_torneo == 1)) else x.seleccionado, axis = 1)
            
        df = df.drop(columns=['fila','en_torneo','min_aic'])
    
    if tamano_elitismo > 0:
        df['dummy'] = 0
        df['orden'] = df.sort_values(['aic']).groupby(['dummy']).cumcount() + 1
        
        df['seleccionado'] = df.apply(lambda x: 1 if x.orden <= tamano_elitismo else x.seleccionado, axis = 1)
        
        df = df.drop(columns=['dummy','orden'])
        
    return(df)

In [9]:
# Generar función iterable

def ga_optimizacion(df
                    , funcion_fitness
                    , sobrevivientes
                    , factor_seleccion
                    , tipo_seleccion
                    , tamaño_torneo
                    , tamaño_elitismo
                    , tipo_cruce
                    , tipo_dato):
    
    